In [45]:
from __future__ import print_function
import datetime
import os.path
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [46]:
import sys
import os
script_dir = os.getcwd()
sys.path.append(script_dir)

In [47]:
SCOPES = ['https://www.googleapis.com/auth/calendar']

In [38]:
def get_credentials():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=8080)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [39]:
def adicionar_evento(service, resumo, local, descricao, inicio, fim):
    evento = {
        'summary': resumo,
        'location': local,
        'description': descricao,
        'start': {
            'dateTime': inicio,
            'timeZone': 'America/Sao_Paulo',
        },
        'end': {
            'dateTime': fim,
            'timeZone': 'America/Sao_Paulo',
        },
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 24 * 60},
                {'method': 'popup', 'minutes': 10},
            ],
        },
    }
    evento = service.events().insert(calendarId='primary', body=evento).execute()
    print(f'Evento criado: {evento.get("htmlLink")}')
    return evento


In [40]:
def buscar_eventos(service, dias_fim):
    agora = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indica UTC
    fim_periodo = (datetime.datetime.utcnow() + datetime.timedelta(days=dias_fim)).isoformat() + 'Z'

    eventos_resultados = service.events().list(
        calendarId='primary',
        timeMin=agora,
        timeMax=fim_periodo,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    eventos = eventos_resultados.get('items', [])

    if not eventos:
        print('Nenhum evento encontrado.')
    for evento in eventos:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        print(inicio, evento['summary'])
    return eventos


In [41]:
def buscar_eventos_por_horas(service, horas_fim):
    agora = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indica UTC
    fim_periodo = (datetime.datetime.utcnow() + datetime.timedelta(hours=horas_fim)).isoformat() + 'Z'

    eventos_resultados = service.events().list(
        calendarId='primary',
        timeMin=agora,
        timeMax=fim_periodo,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    eventos = eventos_resultados.get('items', [])

    if not eventos:
        print('Nenhum evento encontrado.')
    for evento in eventos:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        print(inicio, evento['summary'])
    return eventos


In [42]:
def deletar_evento(service, event_id):
    try:
        service.events().delete(calendarId='primary', eventId=event_id).execute()
        print(f'Evento {event_id} deletado com sucesso.')
    except Exception as e:
        print(f'Erro ao deletar o evento: {e}')

In [51]:
def main():
    creds = get_credentials()
    service = build('calendar', 'v3', credentials=creds)

    # Testando adicionar um evento
    print("Adicionando evento...")
    evento = adicionar_evento(service, 'Reunião de Teste', 'Remoto', 'Reunião de teste via API do Google Calendar.',
                              '2024-07-01T20:00:00-03:00', '2024-07-01T21:00:00-03:00')

    # Testando buscar eventos em um período de 7 dias
    print("\nBuscando eventos para os próximos 7 dias...")
    buscar_eventos(service, 7)

    # Testando buscar eventos em um período de 24 horas
    print("\nBuscando eventos para as próximas 24 horas...")
    buscar_eventos_por_horas(service, 24)

    # Testando deletar um evento
    print("\nDeletando evento adicionado...")
    deletar_evento(service, evento['id'])

In [53]:
main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1173353199-m995knl2ls4bm89bvo1hj7ohk1lt59l2.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=Z9zYSIYmemoECbCJKw6m6hfb59uf7e&access_type=offline
